In [1]:
### Import packages ###
import ast
import itertools
import numpy as np
import pandas as pd

# Input Data Set 

In [2]:
### Regression ###
# Data =["Auto"]
# Data =["BostonHousing"]
# Data =["CPS"]
# Data =["Simulate"]
# Data =["Yacht"]

### Classification ###
Data =["BostonHousingBinned"]


# Input directory

In [3]:
# Directory
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Rashomon Parameters

In [4]:
# Rashomon QBC parameters
Seed = range(0, 50)
Data = ["BostonHousingBinned"]  # Example Data
TestProportion = [0.2]
CandidateProportion = [0.8]
SelectorType = ["TreeEnsembleQBCFunction"]
ModelType = ["RandomForestClassificationFunction", "TreeFarmsFunction"]
DataArgs = [{"N": 1000, "K": 5}]
SelectorArgs = [
    {"TopCModels": 10, "UniqueErrorsInput": True},
    {"TopCModels": 10, "UniqueErrorsInput": False},
    {"TopCModels": 100, "UniqueErrorsInput": True},
    {"TopCModels": 100, "UniqueErrorsInput": False},
]
ModelArgs = [
    {"n_estimators": 100,
     "Seed": 1,
     "TopCModels": 10, 
     "config": {"regularization": 0.01, "rashomon_bound_multiplier": 0.1}, "Type": "Classification"},
    {"n_estimators": 100,
     "Seed": 1,
     "TopCModels": 100, 
     "config": {"regularization": 0.01, "rashomon_bound_multiplier": 0.1}, "Type": "Classification"}
]

# Generate filtered combinations
filtered_combinations = [
    (seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg)
    for seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg in itertools.product(
        Seed, Data, TestProportion, CandidateProportion, SelectorType, ModelType, DataArgs, SelectorArgs, ModelArgs
    )
    if (model_type == "TreeFarmsFunction" and sel_arg["TopCModels"] == mod_arg["TopCModels"]) or
       (model_type == "RandomForestClassificationFunction" and sel_arg["TopCModels"] == mod_arg["TopCModels"])
]

df = pd.DataFrame(filtered_combinations, columns=[
    "Seed", "Data", "TestProportion", "CandidateProportion", "SelectorType", 
    "ModelType", "DataArgs", "SelectorArgs", "ModelArgs"])

df.loc[df["ModelType"] == "RandomForestClassificationFunction", "SelectorArgs"] = "{'TopCModels': 10, 'UniqueErrorsInput': False}"
df.loc[df["ModelType"] == "RandomForestClassificationFunction", "ModelArgs"] = "{'n_estimators': 100, 'Seed': 1, 'TopCModels': 100, 'config': {'regularization': 0.01, 'rashomon_bound_multiplier': 0.1}, 'Type': 'Classification'}"


In [5]:
def make_hashable(value):
    """Convert dictionaries or lists to a hashable representation."""
    if isinstance(value, dict):
        return str(value)  # Convert dictionary to string
    elif isinstance(value, list):
        return tuple(value)  # Convert list to tuple
    return value

df = df.applymap(make_hashable)  # Apply the function to every cell
df = df.drop_duplicates()  # Now drop duplicates
df.index = range(0,df.shape[0])

/var/folders/10/2lfzfs1j0j933_mjkrskp5kh0000gq/T/ipykernel_17119/1600418422.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(make_hashable)  # Apply the function to every cell


# Create JobName and Output

In [6]:
### Job Name ###
ReplacementCharacters = {":": "",  " ": "", "{": "", "}": "", """: "", ",": "_", """: "", "[": "", "]": ""}
DataArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(DataArgs)])
ModelArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
SelectorArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
ModelTypeReplacement = str(ModelType).replace("Function","")
DataReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(Data)])

df["JobName"] = ("Seed" + df["Seed"].astype(str) +
                 "_Data" + df["Data"].astype(str) +
                 "_TP" + df["TestProportion"].astype(str) +
                 "_CP" + df["CandidateProportion"].astype(str) +
                 "_ST" + df["SelectorType"].astype(str) +
                 "_MT" + df["ModelType"].astype(str) + 
                 "_UEI" + df["SelectorArgs"].apply(lambda x: str(ast.literal_eval(x)["UniqueErrorsInput"])) +
                 "_RashomonNum" + df["SelectorArgs"].apply(lambda x: str(ast.literal_eval(x)["TopCModels"]))
                 )


In [7]:
### Output Name ###
df["JobName"] = df["JobName"].str.replace("Function", "", regex=False)
df["Output"] =  df["Data"].astype(str) + "/" + df["ModelType"].astype(str) + "/Raw/" + df["JobName"] + ".pkl"
df["Output"] = df["Output"].str.replace("Function", "", regex=False)


In [8]:
# Define the columns to process
columns_to_process = ['DataArgs', 'SelectorArgs', 'ModelArgs']
for col in columns_to_process:
    df[col] = "'" + df[col].apply(lambda x: str(x).replace("'", '"') if pd.notna(x) else x) + "'"


In [9]:
### Save Parameter Vector ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data[0]) + ".csv"
df.to_csv(output_path, index=False)

In [10]:
len(set(df["JobName"]))

250

In [11]:
len(set(df["Output"]))

250

In [12]:
df.shape

(250, 11)